In [1]:
from pynse import *
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter
#import mplfinance as mpf
import time, asyncio
import matplotlib.dates as mdates
import mplfinance as mpf
import statsmodels.api as sm
import statsmodels.formula.api as smf
from datetime import datetime
import os
import glob
import pandas as pd
import mplfinance as mpf
from matplotlib.dates import MonthLocator, DateFormatter
import scipy.stats as stats
from tvDatafeed import TvDatafeed,Interval
tv=TvDatafeed()


nse=Nse()
#nse.bhavcopy(series="all")
#bhavcopy_full=nse.bhavcopy(series="all")

you are using nologin method, data you access may be limited


LOADING RAW DATA AND SAVING PROCESSED CSV's

In [2]:
#Data for PE and MCAP
folder_path = "C:/Users/99111/OneDrive/Desktop/Full data set/Price, PE, EBITDA , EV & Mcap/*.csv"

files = glob.glob(folder_path)

df_list = []

for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

PEMCAP = pd.concat(df_list)
PEMCAP = PEMCAP.rename(columns={'NDP_Date': 'Date'})
PEMCAP['Date'] = pd.to_datetime(PEMCAP['Date'])
Sectors = PEMCAP['CD_Sector'].unique().tolist()
Industrys = PEMCAP['CD_Industry'].unique().tolist()
PEMCAP = PEMCAP.loc[:, ['CD_NSE Symbol', 'CD_Sector' , 'CD_Industry', 'Date', 'NDP_Mcap', 'NDP_Consolidated PE', 'NDP_Consolidated EV EBIDTA']].copy()
PEMCAP['uniquecode'] = PEMCAP['CD_NSE Symbol'] + PEMCAP['Date'].astype(str)

#Price and symbol data
folder_path = "C:/Users/99111/OneDrive/Documents/pynse/Shashank work/data/raw/Delivdata/*.csv"

files = glob.glob(folder_path)

df_list = []

for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

df = pd.concat(df_list)

df = df.rename(columns={'DATE1': 'Date'})


df['Open'] = np.where(df['OPEN_PRICE'] > 0, df['OPEN_PRICE'], df['OPEN'])
df['High'] = np.where(df['HIGH_PRICE'] > 0, df['HIGH_PRICE'], df['HIGH'])
df['Low'] = np.where(df['LOW_PRICE'] > 0, df['LOW_PRICE'], df['LOW'])
df['Close'] = np.where(df['CLOSE_PRICE'] > 0, df['CLOSE_PRICE'], df['CLOSE'])
df['Prevclose'] = np.where(df['PREV_CLOSE'] > 0, df['PREV_CLOSE'], df['PREVCLOSE']) 

df['TradedQuantity'] = np.where(df['TTL_TRD_QNTY'] > 0, df['TTL_TRD_QNTY'], df['TOTTRDQTY'])
df['Numberoftrades'] = np.where(df['NO_OF_TRADES'] > 0, df['NO_OF_TRADES'], df['TOTALTRADES'])
df['Averageprice'] = np.where(df['AVG_PRICE'] > 0, df['AVG_PRICE'], (df['Open']+df['High']+df['Low']+df['Close']+df['Prevclose'])/5)

df['TURNOVER_LACS'] = df['TURNOVER_LACS'] / 100
df['TOTTRDVAL'] = df['TOTTRDVAL'] / 10000000
df['Turnover_CR'] = np.where(df['TURNOVER_LACS'] > 0, df['TURNOVER_LACS'], df['TOTTRDVAL'])
df = df.loc[:, ['SYMBOL','Date', 'Prevclose', 'Open', 'High', 'Low', 'Close','Averageprice', 'TradedQuantity', 'Numberoftrades', 'Turnover_CR', 'DELIV_QTY', 'DELIV_PER']]
df['uniquecode'] = df['SYMBOL'] + df['Date'].astype(str)
df = df.sort_values(by='Date', ascending=True)

#Combining data
df1 = pd.merge(df, PEMCAP, on='uniquecode', how='left')
df1.drop_duplicates(subset='uniquecode', keep='first', inplace=True)
df1['Date_x'] = pd.to_datetime(df1['Date_x'])
df1 = df1.loc[:, ['SYMBOL','Date_x', 'Prevclose', 'Open', 'High', 'Low', 'Close', 'Averageprice', 'TradedQuantity', 'Numberoftrades', 'Turnover_CR', 'DELIV_QTY', 'CD_Sector', 'CD_Industry', 'NDP_Mcap', 'NDP_Consolidated PE', 'NDP_Consolidated EV EBIDTA']]

#Cutting of data data lenght
datacutoffdate = pd.to_datetime('2010-1-1')
df1 = df1[df1['Date_x'] >= datacutoffdate]

#creating symbols list 
symbolcutoffdate = pd.to_datetime('2023-7-20')
symdf = df1[df1['Date_x'] >= symbolcutoffdate]
symbols = symdf['SYMBOL'].unique().tolist()
df1 = df1[df1['SYMBOL'].isin(symbols)]

######################

#file_path = r'C:\Users\99111\OneDrive\Desktop\Stocklist.xlsx'
#symbols = pd.read_excel(file_path)
#symbols = symbols['Symbol_NIFTY100'].unique().tolist()

Creating CSV's of the raw data and the symbols selected are based on the 2 sets of code block right abpve

In [ ]:

for symbol in symbols:
    try:
        df2=df1[(df1['SYMBOL'] == symbol)].copy()

        df2 = df2.sort_values(by='Date_x', ascending=True)
        df2 = df2.reset_index(drop=True)

        cols_to_fill_forward = ['CD_Sector', 'CD_Industry', 'NDP_Mcap', 'NDP_Consolidated PE', 'NDP_Consolidated EV EBIDTA']
        df2[cols_to_fill_forward] = df2[cols_to_fill_forward].fillna(method='ffill')

        #fields for Industry analysis
        df2['Return_3days'] = ((df2['NDP_Mcap'] - df2['NDP_Mcap'].shift(3)) / df2['NDP_Mcap'].shift(3)) * 100
        df2['Return_5days'] = ((df2['NDP_Mcap'] - df2['NDP_Mcap'].shift(5)) / df2['NDP_Mcap'].shift(5)) * 100

        df2['TurnoverCR_3daysAVG'] = df2['Turnover_CR'].rolling(3).mean()
        df2['TO_Weighted_3dayreturns'] = df2['TurnoverCR_3daysAVG'] * df2['Return_3days']
        df2['MCAP_Weighted_3dayreturns'] = df2['NDP_Mcap'] * df2['Return_3days']

        df2['TurnoverCR_5daysAVG'] = df2['Turnover_CR'].rolling(5).mean()
        df2['TO_Weighted_5dayreturns'] = df2['TurnoverCR_5daysAVG'] * df2['Return_5days']
        df2['MCAP_Weighted_5dayreturns'] = df2['NDP_Mcap'] * df2['Return_5days']

        for N in [63, 126, 252, 504]:
            weighting_factors = [i / sum(range(1, N + 1)) for i in range(1, N + 1)]
            df2[f'TWClose_{N}'] = df2['NDP_Mcap'].rolling(window=N, min_periods=N).apply(lambda x: (x * weighting_factors).sum())
    

        df2['Price>TW63'] = (df2['NDP_Mcap'] > df2['TWClose_63']).astype(int)
        df2['Price>TW126'] = (df2['NDP_Mcap'] > df2['TWClose_126']).astype(int)
        df2['Price>TW252'] = (df2['NDP_Mcap'] > df2['TWClose_252']).astype(int)
        df2['Price>TW504'] = (df2['NDP_Mcap'] > df2['TWClose_504']).astype(int)

        output_directory = r'C:\Users\99111\OneDrive\Desktop\daily updates\MarketBreath and StageAnalysis\Data\IndustryData'
        output_filename = f'{symbol}.csv'
        output_path = os.path.join(output_directory, output_filename)

        df2.to_csv(output_path, index=False)

    except Exception as e:
                print(f"Error while processing outer forloop {symbol}: {e}")
                continue

LOADING CSV's

In [ ]:
folder_path = "C:/Users/99111/OneDrive/Desktop/daily updates/MarketBreath and StageAnalysis/Data/IndustryData/*.csv"

files = glob.glob(folder_path)

df_list = []

for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

df = pd.concat(df_list)
Sectors = df['CD_Sector'].unique().tolist()
df['Price>TW63'] = df['Price>TW63'].astype(int)
df['Price>TW126'] = df['Price>TW126'].astype(int)
df['Price>TW252'] = df['Price>TW252'].astype(int)
df['Price>TW504'] = df['Price>TW504'].astype(int)

Defensive and Expantionary Mcap

In [ ]:
defensive_industries = [
    'Pharmaceuticals & Drugs', 'Healthcare Services', 'Hospital & Healthcare Services', 'Power Generation/Distribution',
    'Gas Transmission/Marketing', 'Edible Oil', 'Telecommunication - Service Provider', 'Consumer Food',
    'Telecommunication - Equipment', 'Household & Personal Products', 'Tobacco/Cigarettes', 'Utilities',
    'Food and Beverage', 'Non-Cyclical Consumer Goods'
]

df['defnondef'] = df['CD_Industry'].apply(lambda industry: 'Defensive' if industry in defensive_industries else 'NonDefensive')
DefnonDefs = df['defnondef'].unique().tolist()


df1 = df[df['defnondef'] == 'NonDefensive']
df1 = df1.sort_values(by='Date_x', ascending=True)
df1['Count'] = 1
selected_columns = ['TurnoverCR_3daysAVG', 'NDP_Mcap', 'TO_Weighted_3dayreturns', 'MCAP_Weighted_3dayreturns', 'Count', 'Price>TW63', 'Price>TW126', 'Price>TW252', 'Price>TW504']
grouped_df1 = df1.groupby('Date_x')[selected_columns].sum().reset_index()

df2 = df[df['defnondef'] == 'Defensive']
df2 = df2.sort_values(by='Date_x', ascending=True)
df2['Count'] = 1
selected_columns = ['TurnoverCR_3daysAVG', 'NDP_Mcap', 'TO_Weighted_3dayreturns', 'MCAP_Weighted_3dayreturns', 'Count', 'Price>TW63', 'Price>TW126', 'Price>TW252', 'Price>TW504']
grouped_df2 = df2.groupby('Date_x')[selected_columns].sum().reset_index()

grouped_df1['Date_x'] = pd.to_datetime(grouped_df1['Date_x'])
grouped_df2['Date_x'] = pd.to_datetime(grouped_df2['Date_x'])


date = pd.to_datetime('2012-7-1')
grouped_df1 = grouped_df1[grouped_df1['Date_x'] >= date]
grouped_df2 = grouped_df2[grouped_df2['Date_x'] >= date]

fig, ax1 = plt.subplots(figsize=(20, 10))

        
ax1.set_xlabel('Date')
ax1.set_ylabel('Def')
ax1.plot(grouped_df1['Date_x'], grouped_df1['NDP_Mcap'], label='Expansionary', color = 'black')
ax1.tick_params(axis='y')
ax1.legend(loc='upper left')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

        
#ax2.set_ylabel(DefnonDef)  # we already handled the x-label with ax1
ax2.plot(grouped_df2['Date_x'], grouped_df2['NDP_Mcap'], label='Defensive')
ax2.tick_params(axis='y')
ax2.legend(loc='upper right')

fig.tight_layout()

save_dir = r'C:\Users\99111\OneDrive\Desktop\daily updates\MarketBreath and StageAnalysis'        
chart_file = os.path.join(save_dir, "Expansive and Defensive.jpg")
plt.savefig(chart_file, dpi=100)

# Close the figure to release resources
plt.close(fig)


Marketwide and Industries Market breath

In [ ]:
df['Date_x'] = pd.to_datetime(df['Date_x'])
df = df.sort_values(by='Date_x', ascending=True)
df['Count'] = 1
selected_columns = ['TurnoverCR_3daysAVG', 'NDP_Mcap', 'TO_Weighted_3dayreturns', 'MCAP_Weighted_3dayreturns', 'Count', 'Price>TW63', 'Price>TW126', 'Price>TW252', 'Price>TW504']
grouped_df = df.groupby('Date_x')[selected_columns].sum().reset_index()
grouped_df['McapWeightedReturn'] = grouped_df['MCAP_Weighted_3dayreturns'] / grouped_df['NDP_Mcap']
grouped_df['TOWeightedReturn'] = grouped_df['TO_Weighted_3dayreturns'] / grouped_df['TurnoverCR_3daysAVG']

grouped_df['Over63'] = (grouped_df['Price>TW63'] / grouped_df['Count']) * 100
grouped_df['Over126'] = (grouped_df['Price>TW126'] / grouped_df['Count']) * 100
grouped_df['Over252'] = (grouped_df['Price>TW252'] / grouped_df['Count']) * 100
grouped_df['Over504'] = (grouped_df['Price>TW504'] / grouped_df['Count']) * 100

grouped_df['63-504'] = grouped_df['Price>TW63'] - grouped_df['Price>TW504']
grouped_df['126-504'] = grouped_df['Price>TW63'] - grouped_df['Price>TW252']


grouped_df['Date_x'] = pd.to_datetime(grouped_df['Date_x'])

date = pd.to_datetime('2020-7-1')
Marketwide = grouped_df[grouped_df['Date_x'] >= date]

fig, axs = plt.subplots(2, 1, figsize=(20, 10))
# Create the first subplot for columns A and B
axs[0].plot(Marketwide['Date_x'],Marketwide['Over63'], label='Over63')
axs[0].plot(Marketwide['Date_x'],Marketwide['Over252'], label='Over252')
axs[0].plot(Marketwide['Date_x'],Marketwide['Over504'], label='Over504')
axs[0].legend()
axs[0].axhline(35, color='black')
axs[0].axhline(60, color='black')
axs[0].set_title('MarketWide')
axs[0].xaxis.set_major_locator(mdates.MonthLocator(interval=2))
axs[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
   
ax2 = axs[1].twinx()
axs[1].plot(Marketwide['Date_x'],Marketwide['63-504'], label='63-504', color='green')
axs[1].plot(Marketwide['Date_x'],Marketwide['126-504'], label='126-504', color='red')
axs[1].axhline(0, color='black')
# Format x-axis date ticks for the second subplot
axs[1].xaxis.set_major_locator(mdates.MonthLocator(interval=2))
axs[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
#ax2.plot(grouped_df1['CE.totalSellQuantity'], label='call SQ', color='red')
#ax2.plot(grouped_df1['PE.totalSellQuantity'], label='Put SQ', color='green')

# Set labels and titles for both y-axes
axs[1].set_ylabel('Call TSQ / Put TSQ')

# Combine the legends from both axes
lines, labels = axs[1].get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
axs[1].legend(lines + lines2, labels + labels2)

axs[1].set_title('MarketWide')

# Adjust the spacing to avoid overlapping labels
fig.tight_layout()
save_dir = r'C:\Users\99111\OneDrive\Desktop\daily updates\MarketBreath and StageAnalysis\Industry Charts'        
chart_file = os.path.join(save_dir, f"{'MarketWide'}.jpg")
plt.savefig(chart_file, dpi=100)

# Close the figure to release resources
plt.close(fig)


for sector in Sectors:
    try:
           
        df1 = df[df['CD_Sector'] == sector]
        df1 = df1.sort_values(by='Date_x', ascending=True)
        df1['Count'] = 1
        selected_columns = ['TurnoverCR_3daysAVG', 'NDP_Mcap', 'TO_Weighted_3dayreturns', 'MCAP_Weighted_3dayreturns', 'Count', 'Price>TW63', 'Price>TW126', 'Price>TW252', 'Price>TW504']
        grouped_df = df1.groupby('Date_x')[selected_columns].sum().reset_index()
        grouped_df['McapWeightedReturn'] = grouped_df['MCAP_Weighted_3dayreturns'] / grouped_df['NDP_Mcap']
        grouped_df['TOWeightedReturn'] = grouped_df['TO_Weighted_3dayreturns'] / grouped_df['TurnoverCR_3daysAVG']

        grouped_df['Over63'] = (grouped_df['Price>TW63'] / grouped_df['Count']) * 100
        grouped_df['Over126'] = (grouped_df['Price>TW126'] / grouped_df['Count']) * 100
        grouped_df['Over252'] = (grouped_df['Price>TW252'] / grouped_df['Count']) * 100
        grouped_df['Over504'] = (grouped_df['Price>TW504'] / grouped_df['Count']) * 100

        grouped_df['63-504'] = grouped_df['Price>TW63'] - grouped_df['Price>TW504']
        grouped_df['126-504'] = grouped_df['Price>TW63'] - grouped_df['Price>TW252']

        grouped_df['Date_x'] = pd.to_datetime(grouped_df['Date_x'])

        #date = pd.to_datetime('2020-7-1')
        grouped_df1 = grouped_df[grouped_df['Date_x'] >= date]

        fig, axs = plt.subplots(2, 1, figsize=(30, 15))
        # Create the first subplot for columns A and B
        axs[0].plot(grouped_df1['Date_x'],grouped_df1['Over63'], label='Over63')
        axs[0].plot(grouped_df1['Date_x'],grouped_df1['Over252'], label='Over252')
        axs[0].plot(grouped_df1['Date_x'],grouped_df1['Over504'], label='Over504')
        axs[0].plot(Marketwide['Date_x'],Marketwide['Over252'], label='Over252', color='black', linestyle='--')
        axs[0].legend()
        axs[0].axhline(35, color='black')
        axs[0].axhline(60, color='black')
        axs[0].set_title(sector)
        # Format x-axis date ticks
        axs[0].xaxis.set_major_locator(mdates.MonthLocator(interval=2))
        axs[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
                
        ax2 = axs[1].twinx()
        axs[1].plot(grouped_df1['Date_x'],grouped_df1['63-504'], label='63-504', color='green')
        axs[1].plot(grouped_df1['Date_x'],grouped_df1['126-504'], label='126-504', color='red')
        #axs[1].plot(Marketwide['Date_x'],Marketwide['126-504'], label='126-504', color='red')
        axs[1].axhline(0, color='black')
        # Format x-axis date ticks for the second subplot
        axs[1].xaxis.set_major_locator(mdates.MonthLocator(interval=2))
        axs[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        #ax2.plot(grouped_df1['CE.totalSellQuantity'], label='call SQ', color='red')
        #ax2.plot(grouped_df1['PE.totalSellQuantity'], label='Put SQ', color='green')

        # Set labels and titles for both y-axes
        axs[1].set_ylabel('Call TSQ / Put TSQ')

        # Combine the legends from both axes
        lines, labels = axs[1].get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        axs[1].legend(lines + lines2, labels + labels2)

        axs[1].set_title(sector)

        # Adjust the spacing to avoid overlapping labels
        fig.tight_layout()
        save_dir = r'C:\Users\99111\OneDrive\Desktop\daily updates\MarketBreath and StageAnalysis\Industry Charts'        
        chart_file = os.path.join(save_dir, f"{sector}.jpg")
        plt.savefig(chart_file, dpi=100)

        # Close the figure to release resources
        plt.close(fig)


    except Exception as e:
                print(f"Error while processing outer forloop {sector}: {e}")
                continue


Expansionary and Defensive Market breath

In [ ]:


for sector in DefnonDefs:
    try:
           
        df1 = df[df['defnondef'] == sector]
        df1 = df1.sort_values(by='Date_x', ascending=True)
        df1['Count'] = 1
        selected_columns = ['TurnoverCR_3daysAVG', 'NDP_Mcap', 'TO_Weighted_3dayreturns', 'MCAP_Weighted_3dayreturns', 'Count', 'Price>TW63', 'Price>TW126', 'Price>TW252', 'Price>TW504']
        grouped_df = df1.groupby('Date_x')[selected_columns].sum().reset_index()
        grouped_df['McapWeightedReturn'] = grouped_df['MCAP_Weighted_3dayreturns'] / grouped_df['NDP_Mcap']
        grouped_df['TOWeightedReturn'] = grouped_df['TO_Weighted_3dayreturns'] / grouped_df['TurnoverCR_3daysAVG']

        grouped_df['Over63'] = (grouped_df['Price>TW63'] / grouped_df['Count']) * 100
        grouped_df['Over126'] = (grouped_df['Price>TW126'] / grouped_df['Count']) * 100
        grouped_df['Over252'] = (grouped_df['Price>TW252'] / grouped_df['Count']) * 100
        grouped_df['Over504'] = (grouped_df['Price>TW504'] / grouped_df['Count']) * 100

        grouped_df['63-504'] = grouped_df['Price>TW63'] - grouped_df['Price>TW504']
        grouped_df['126-504'] = grouped_df['Price>TW63'] - grouped_df['Price>TW252']

        grouped_df['Date_x'] = pd.to_datetime(grouped_df['Date_x'])

        #date = pd.to_datetime('2020-7-1')
        grouped_df1 = grouped_df[grouped_df['Date_x'] >= date]

        fig, axs = plt.subplots(2, 1, figsize=(30, 15))
        # Create the first subplot for columns A and B
        axs[0].plot(grouped_df1['Date_x'],grouped_df1['Over63'], label='Over63')
        axs[0].plot(grouped_df1['Date_x'],grouped_df1['Over252'], label='Over252')
        axs[0].plot(grouped_df1['Date_x'],grouped_df1['Over504'], label='Over504')
        axs[0].plot(Marketwide['Date_x'],Marketwide['Over252'], label='Over252', color='black', linestyle='--')
        axs[0].legend()
        axs[0].axhline(35, color='black')
        axs[0].axhline(60, color='black')
        axs[0].set_title(sector)
        # Format x-axis date ticks
        axs[0].xaxis.set_major_locator(mdates.MonthLocator(interval=2))
        axs[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
                
        ax2 = axs[1].twinx()
        axs[1].plot(grouped_df1['Date_x'],grouped_df1['63-504'], label='63-504', color='green')
        axs[1].plot(grouped_df1['Date_x'],grouped_df1['126-504'], label='126-504', color='red')
        #axs[1].plot(Marketwide['Date_x'],Marketwide['126-504'], label='126-504', color='red')
        axs[1].axhline(0, color='black')
        # Format x-axis date ticks for the second subplot
        axs[1].xaxis.set_major_locator(mdates.MonthLocator(interval=2))
        axs[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        #ax2.plot(grouped_df1['CE.totalSellQuantity'], label='call SQ', color='red')
        #ax2.plot(grouped_df1['PE.totalSellQuantity'], label='Put SQ', color='green')

        # Set labels and titles for both y-axes
        axs[1].set_ylabel('Call TSQ / Put TSQ')

        # Combine the legends from both axes
        lines, labels = axs[1].get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        axs[1].legend(lines + lines2, labels + labels2)

        axs[1].set_title(sector)

        # Adjust the spacing to avoid overlapping labels
        fig.tight_layout()
        save_dir = r'C:\Users\99111\OneDrive\Desktop\daily updates\MarketBreath and StageAnalysis'        
        chart_file = os.path.join(save_dir, f"{sector}.jpg")
        plt.savefig(chart_file, dpi=100)

        # Close the figure to release resources
        plt.close(fig)


    except Exception as e:
                print(f"Error while processing outer forloop {sector}: {e}")
                continue